In [ ]:
#Natural Language Processing

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [ ]:
nltk.help.upenn_tagset() #PenTree

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [ ]:
sentence=word_tokenize("Third wave of corona virus is here")
nltk.pos_tag(sentence)

[('Third', 'JJ'),
 ('wave', 'NN'),
 ('of', 'IN'),
 ('corona', 'NN'),
 ('virus', 'NN'),
 ('is', 'VBZ'),
 ('here', 'RB')]

In [ ]:
#n GRAMS IN SENTIMENT ANALYSIS

import numpy as np
import pandas as pd
from sklearn import model_selection, naive_bayes, svm

In [ ]:
df=pd.read_csv("/content/sample_data/Reviews.csv",engine='python',error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 18282: unexpected end of data


In [ ]:
df.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [ ]:
df.shape

(18280, 10)

In [ ]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Score'].value_counts()

5    11466
4     2630
1     1631
3     1532
2     1021
Name: Score, dtype: int64

In [ ]:
df['positively rated']=np.where(df['Score']>=3,1,0)

In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,positively rated
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [ ]:
df['positively rated'].value_counts()

1    15628
0     2652
Name: positively rated, dtype: int64

In [ ]:
#Highly imbalanced
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df["Text"],df["positively rated"],random_state=50)

In [ ]:
print(X_train)

8139     I was very unhappy when this package arrived a...
4164     Been gluten free for 7 years now. Thought we w...
5787     I use these vanilla beans to make my infamous ...
16483    These crackers taste absolutely delicious, but...
11555    Salt tastes fine, cant say about the health cl...
                               ...                        
14565    My husband and I always a cup of Big Easy Bold...
15649    My 2 year old German Shepherd loves this treat...
10123    I have not had pork rinds in more than 20 year...
5600     Very good coffee.  Can't always find it in our...
14000    My Malamutes aren't too picky, except one who ...
Name: Text, Length: 13710, dtype: object


In [ ]:
print(y_train)

8139     0
4164     0
5787     1
16483    0
11555    1
        ..
14565    1
15649    1
10123    1
5600     1
14000    1
Name: positively rated, Length: 13710, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect=CountVectorizer(min_df=5,ngram_range=(1,2),).fit(X_train)

In [ ]:
len(vect.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


35704

In [ ]:
X_train_vectorized=vect.transform(X_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train_vectorized,y_train)

MultinomialNB()

In [ ]:
predictions=model.predict(vect.transform(X_test))

In [ ]:
print(predictions)

[1 1 1 ... 1 1 0]


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[ 469  214]
 [ 222 3665]]
              precision    recall  f1-score   support

           0       0.68      0.69      0.68       683
           1       0.94      0.94      0.94      3887

    accuracy                           0.90      4570
   macro avg       0.81      0.81      0.81      4570
weighted avg       0.91      0.90      0.90      4570



In [ ]:
from sklearn.metrics import roc_auc_score
print("AUC score is",roc_auc_score(y_test,predictions))

AUC score is 0.814781486209428
